# Exporing aviation accidents data

Link to kagge: https://www.kaggle.com/khsamaha/aviation-accident-database-synopses

## Dataset purpose
The declared purpose for publishing the dataset was:
> * Which is the type accident often to happen? Which are the features relevant?
* What is season that there are more accident?
* The amateur have a influence on accident or injury severity?
* Do they take too long to make preliminary reports?
* What do scheme have more accident?
* Where are there more accident? - deprecated
* What do aircraft have more accident? -deprecated
* How do accidents evolve in the time of aviation in the United States?

Credits:
* I took some interesting functions from https://www.kaggle.com/helgejo/titanic/an-interactive-data-science-tutorial


In [2]:
# Python libraries
import math
import re
import datetime


# Handle table-like data and matrices
import numpy as np
import pandas as pd

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# Modelling Helpers
from sklearn.preprocessing import StandardScaler, Imputer , Normalizer , scale
from sklearn.cross_validation import train_test_split , StratifiedKFold
from sklearn.feature_selection import RFECV

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

# Configure visualisations
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6


/opt/conda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Plot and data study helpers

In [3]:
def plot_histograms( df , variables , n_rows , n_cols ):
    fig = plt.figure( figsize = ( 16 , 12 ) )
    for i, var_name in enumerate( variables ):
        ax=fig.add_subplot( n_rows , n_cols , i+1 )
        df[ var_name ].hist( bins=10 , ax=ax )
        ax.set_title( 'Skew: ' + str( round( float( df[ var_name ].skew() ) , ) ) ) # + ' ' + var_name ) #var_name+" Distribution")
        ax.set_xticklabels( [] , visible=False )
        ax.set_yticklabels( [] , visible=False )
    fig.tight_layout()  # Improves appearance a bit.
    plt.show()

def plot_distribution( df , var , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , hue=target , aspect=4 , row = row , col = col )
    facet.map( sns.kdeplot , var , shade= True )
    facet.set( xlim=( 0 , df[ var ].max() ) )
    facet.add_legend()

def plot_categories( df , cat , target , **kwargs ):
    row = kwargs.get( 'row' , None )
    col = kwargs.get( 'col' , None )
    facet = sns.FacetGrid( df , row = row , col = col )
    facet.map( sns.barplot , cat , target )
    facet.add_legend()

def plot_correlation_map( df ):
    corr = titanic.corr()
    _ , ax = plt.subplots( figsize =( 12 , 10 ) )
    cmap = sns.diverging_palette( 220 , 10 , as_cmap = True )
    _ = sns.heatmap(
        corr, 
        cmap = cmap,
        square=True, 
        cbar_kws={ 'shrink' : .9 }, 
        ax=ax, 
        annot = True, 
        annot_kws = { 'fontsize' : 12 }
    )

def describe_more( df ):
    var = [] ; l = [] ; t = []
    for x in df:
        var.append( x )
        l.append( len( pd.value_counts( df[ x ] ) ) )
        t.append( df[ x ].dtypes )
    levels = pd.DataFrame( { 'Variable' : var , 'Levels' : l , 'Datatype' : t } )
    levels.sort_values( by = 'Levels' , inplace = True )
    return levels

def plot_variable_importance( X , y ):
    tree = DecisionTreeClassifier( random_state = 99 )
    tree.fit( X , y )
    plot_model_var_imp( tree , X , y )
    
def plot_model_var_imp( model , X , y ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh' )
    print (model.score( X , y ))
    
def category_values(dataframe, categories):
    for c in categories:
        print('\n', dataframe.groupby(by=c)[c].count().sort_values(ascending=False))
        print('Nulls: ', dataframe[c].isnull().sum())


## Loading data

In [4]:
df = pd.read_csv('AviationDataEnd2016UP.csv', sep=',', header=0, encoding = 'iso-8859-1')

df.sample(10)

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
22563,20040729X01091,Accident,FTW04CA140,2004-05-26,"Oklahoma City, OK",United States,35.790555,-97.940555,2DT,Downtown Airpark,...,Personal,NaN,NaN,NaN,1.0,NaN,VMC,LANDING,Probable Cause,29/09/2004
49160,20001211X14451,Accident,LAX92LA168,1992-04-04,"CARLSBAD, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,0.0,1.0,1.0,VMC,LANDING,Probable Cause,26/08/1993
68382,20001214X35704,Accident,FTW85LA121,1985-02-19,"HOLMWOOD, LA",United States,NaN,NaN,NaN,NaN,...,Positioning,NaN,0.0,0.0,1.0,0.0,IMC,CRUISE,Probable Cause,NaN
64150,20001213X33962,Accident,MIA86LA232,1986-06-29,"OCALA, FL",United States,NaN,NaN,OCF,OKALA MUNICIPAL,...,Personal,NaN,0.0,0.0,0.0,2.0,VMC,LANDING,Probable Cause,NaN
57214,20001213X27365,Accident,ANC89LA025,1988-12-21,"KOTZEBUE, AK",United States,NaN,NaN,OTZ,WIEN MEMORIAL,...,Unknown,BAKER AVIATION INC.,0.0,0.0,0.0,6.0,IMC,LANDING,Probable Cause,06/12/1989
30233,20001212X21633,Accident,ANC00LA101,2000-08-11,"CENTRAL, AK",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,0.0,1.0,0.0,VMC,TAKEOFF,Probable Cause,18/05/2001
49866,20001212X18406,Accident,ANC92LA016,1991-11-08,"STEBBINS, AK",United States,NaN,NaN,WBB,STEBBINS,...,Business,NaN,0.0,0.0,0.0,2.0,VMC,LANDING,Probable Cause,09/03/1993
51080,20001212X17339,Accident,MIA91LA161,1991-06-12,"JACKSONVILLE, FL",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,0.0,0.0,0.0,3.0,VMC,CRUISE,Probable Cause,10/03/1993
57175,20001213X27401,Incident,DCA89IA016,1988-12-26,"MORGANTOWN, WV",United States,NaN,NaN,NaN,NaN,...,Unknown,"EASTERN AIR LINES, INC. (DBA: EASTERN AIR LINES)",0.0,0.0,2.0,108.0,VMC,CRUISE,Probable Cause,11/06/1990
13053,20090414X14441,Accident,WPR09CA193,2009-04-13,"Dubois, WY",United States,43.752222,-110.067223,NaN,NaN,...,Personal,NaN,NaN,NaN,2.0,NaN,VMC,CRUISE,Probable Cause,14/07/2009


### Getting info on the fields types

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79293 entries, 0 to 79292
Data columns (total 31 columns):
Event.Id                  79293 non-null object
Investigation.Type        79293 non-null object
Accident.Number           79293 non-null object
Event.Date                79293 non-null object
Location                  79215 non-null object
Country                   78786 non-null object
Latitude                  25751 non-null float64
Longitude                 25742 non-null float64
Airport.Code              44666 non-null object
Airport.Name              47439 non-null object
Injury.Severity           79293 non-null object
Aircraft.Damage           76883 non-null object
Aircraft.Category         22477 non-null object
Registration.Number       76209 non-null object
Make                      79204 non-null object
Model                     79175 non-null object
Amateur.Built             78721 non-null object
Number.of.Engines         75175 non-null float64
Engine.Type             

Let's see what kind of numeric data we have

In [6]:
df.describe()

/opt/conda/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Latitude,Longitude,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,25751.000000,25742.000000,75175.000000,55984.000000,53742.000000,54833.000000,66949.000000
mean,37.690421,-93.781061,1.148055,0.814679,0.317703,0.502581,5.790886
std,12.148019,39.243662,0.453847,6.233700,1.372924,2.781994,29.223016
min,-78.016945,-178.676111,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,89.218056,177.557778,18.000000,349.000000,111.000000,380.000000,699.000000


### Getting some counts on how many different values are there for each feature

In [7]:
describe_more(df)

,Datatype,Levels,Variable
1,object,2,Investigation.Type
16,object,2,Amateur.Built
27,object,3,Weather.Condition
20,object,3,Schedule
11,object,3,Aircraft.Damage
29,object,4,Report.Status
17,float64,6,Number.of.Engines
28,object,12,Broad.Phase.of.Flight
12,object,13,Aircraft.Category
18,object,14,Engine.Type


In [8]:
# splitting date field in the components

df['Year'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").year)
df['Month'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").month)
df['Day'] = df['Event.Date'].apply(lambda d: datetime.datetime.strptime(d, "%Y-%m-%d").day)


### Looking at some categories
I try to list some unique values in the categories fields to subsequently plot some data distribution over those.

In [9]:
categories = ['Investigation.Type',
             'Aircraft.Damage',
             'Aircraft.Category',
             'Amateur.Built',
             'Number.of.Engines',
             'Engine.Type',
             'FAR.Description',
             'Schedule',
             'Purpose.of.Flight',
             'Weather.Condition',
             'Broad.Phase.of.Flight',
             'Report.Status']

for c in categories:
    print(c , df[c].unique())


Investigation.Type ['Accident' 'Incident']
Aircraft.Damage ['Substantial' 'Destroyed' nan 'Minor']
Aircraft.Category ['Airplane' 'Helicopter' 'Weight-Shift' 'Glider' 'Unknown' 'Balloon'
 'Powered Parachute' 'Ultralight' 'Gyroplane' 'Gyrocraft' nan
 'Powered-Lift' 'Rocket' 'Blimp']
Amateur.Built ['Yes' 'No' nan]
Number.of.Engines [ nan   1.   2.   0.   4.   3.  18.]
Engine.Type ['Reciprocating' nan 'Turbo Prop' 'Turbo Fan' 'Turbo Shaft' 'Unknown'
 'Turbo Jet' 'Electric' 'REC, ELEC' 'None' 'TF, TJ' 'Hybrid Rocket'
 'REC, TJ, TJ' 'REC, TJ, REC, TJ' 'TJ, REC, REC, TJ']
FAR.Description ['Part 91: General Aviation' nan 'Part 135: Air Taxi & Commuter'
 'Public Aircraft' 'Part 121: Air Carrier' 'Unknown'
 'Non-U.S., Non-Commercial' 'Part 137: Agricultural' 'Non-U.S., Commercial'
 'Part 103: Ultralight' 'Part 133: Rotorcraft Ext. Load' 'Public Use'
 'Part 129: Foreign' 'Armed Forces' 'Part 437: Commercial Space Flight'
 'Part 91 Subpart K: Fractional' 'Part 125: 20+ Pax,6000+ lbs'
 'Part 91F: S

### Counting the number of different values for each category feature

In [10]:
category_values(df, categories)



 Investigation.Type
Accident    76118
Incident     3175
Name: Investigation.Type, dtype: int64
Nulls:  0

 Aircraft.Damage
Substantial    57049
Destroyed      17322
Minor           2512
Name: Aircraft.Damage, dtype: int64
Nulls:  2410

 Aircraft.Category
Airplane             19273
Helicopter            2360
Glider                 381
Balloon                175
Gyrocraft              100
Weight-Shift            66
Powered Parachute       48
Unknown                 32
Ultralight              31
Powered-Lift             5
Blimp                    3
Gyroplane                2
Rocket                   1
Name: Aircraft.Category, dtype: int64
Nulls:  56816

 Amateur.Built
No     71105
Yes     7616
Name: Amateur.Built, dtype: int64
Nulls:  572

 Number.of.Engines
1.0     63082
2.0     10057
0.0      1143
3.0       477
4.0       415
18.0        1
Name: Number.of.Engines, dtype: int64
Nulls:  4118

 Engine.Type
Reciprocating       64598
Turbo Shaft          3305
Turbo Prop           3042
Turbo 

## Filling Null values

In [11]:
# null damages can't be defined
df[df['Aircraft.Damage'].isnull()]
df['Aircraft.Damage'].fillna('Unknown', inplace=True)

In [12]:
# null countries are outside US
df[df['Country'].isnull()]
df['Country'].fillna('Foreign', inplace=True)

In [13]:
# null categories can't be defined
df[df['Aircraft.Category'].isnull()]
df['Aircraft.Category'].fillna('Unknown', inplace=True)

#### Amateur producers
Instead of putting an 'unknown' value in the Amateur.Built field, I've collected all the producers and all the amateurs brands/names from the rest of the dataset and filled the null cells searching in the resulting two lists. For the remaining marks that are not present anywhere in the dataset I chose to set them as amateurs.

#### Number of engines
For the balloons I'll set this value to 0.
For the remaining, I'll make some assumptions and aproximations based on the rest of the values.


In [35]:
# Extracting producers and amateurs
producers = [x for x in df['Make'][df['Amateur.Built']== 'No'].unique() ]
amateurs  = [x for x in df['Make'][df['Amateur.Built']== 'Yes'].unique() ]

# -----------------------------------------------
# Function that fixes the null in amateur.built
def fix_amateur_built(ab, m):
    if type(ab) == str:
        return ab
    else:
        if m in producers:
            return 'No'
        else:
            return 'Yes'
# Fix for Amateur.Built field      
am_built = df.apply(lambda x: fix_amateur_built(x['Amateur.Built'], x['Make']), axis=1)
df = df.assign(AmateurBuilt = am_built, index=df.index)

# -----------------------------------------------
# Function that fixes the null in number.of.engines
def fix_number_of_engines(noe, m):
    if noe >= 0:
        return noe
    else:
        # Setting number of engines at the mean number of engines for the producer
        r = np.round(df['Number.of.Engines'][df['Make']==m].mean())
        return r

# Setting 0 engines for balloons
df['Number.of.Engines'][df['Number.of.Engines'].isnull() & (df['Make'].str.contains('balloon', case=False))] = 0.0
# Correcting number of engines
num_engines = df.apply(lambda x: fix_number_of_engines(x['Number.of.Engines'], x['Make']), axis=1)
df = df.assign(NumberofEngines = num_engines, index=df.index)
# Still some null after number of engines correction
df['NumberofEngines'].fillna(1, inplace=True)

# -----------------------------------------------

# Function that fixes the engine types
def fix_engine_type(et, model):
    if type(et) == str:
        return et
    else:
        # Setting number of engines at the mean number of engines for the producer
        e = (df['Engine.Type'][df['Model']==model].mode())
        return e[0] if type(e)==np.ndarray else e
# Fix for Engine.Type field      
en_type = df.apply(lambda x: fix_engine_type(x['Engine.Type'], x['Model']), axis=1)
df = df.assign(EngineType = en_type, index=df.index)


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Checking if all nulls have been fixed

In [36]:
#category_values(df, ['AmateurBuilt', 'NumberofEngines', 'EngineType'])
df['EngineType'].sample(10)

44623    Reciprocating
67088          Unknown
5804     Reciprocating
21636    Reciprocating
4074                []
21009    Reciprocating
77942        Turbo Jet
53904    Reciprocating
21389    Reciprocating
7567         [Unknown]
Name: EngineType, dtype: object